In [ ]:
!pip install pydub
!pip install nltk
!pip install num2words
!pip install word2number
!pip install difflib

ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [ ]:
import torch
import pandas as pd
from pydub import AudioSegment
import os
from nltk.tokenize import sent_tokenize
from difflib import SequenceMatcher
import nltk
import re
from word2number import w2n
from num2words import num2words

# Diccionario para conversiones específicas
number_words = {
    "cero": "0", "uno": "1", "dos": "2", "tres": "3", "cuatro": "4", "cinco": "5", "seis": "6",
    "siete": "7", "ocho": "8", "nueve": "9", "diez": "10", "once": "11", "doce": "12", "trece": "13",
    "catorce": "14", "quince": "15", "dieciséis": "16", "diecisiete": "17", "dieciocho": "18", "diecinueve": "19",
    "veinte": "20", "treinta": "30", "cuarenta": "40", "cincuenta": "50", "sesenta": "60", "setenta": "70",
    "ochenta": "80", "noventa": "90", "cien": "100", "mil": "1000", "millón": "1000000"
}

months = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04", "mayo": "05", "junio": "06",
    "julio": "07", "agosto": "08", "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
}

# Función para convertir números a palabras
#def convert_number_to_words(match):
#    number = match.group(0)
#    if "." in number:  # Para decimales
#        integer, decimal = number.split(".")
#        integer_word = num2words(int(integer), lang='es')
#        decimal_word = " ".join([num2words(int(digit), lang='es') for digit in decimal])
#        return f"{integer_word} punto {decimal_word}"
#    elif len(number) == 4:  # Para años
#        return num2words(int(number), lang='es')
#    else:  # Para números generales
#        return num2words(int(number), lang='es')

def convert_number_to_words(match):
    number = match.group(0)
    if "." in number:  # Para decimales
        integer, decimal = number.split(".")
        integer_word = num2words(int(integer), lang='es')
        decimal_word = " ".join([num2words(int(digit), lang='es') for digit in decimal])
        return f"{integer_word} punto {decimal_word}"
    elif len(number) == 4:  # Para años
        return num2words(int(number), lang='es')
    else:  # Para números generales
        return num2words(int(number), lang='es')

def preprocess_text(text):
    # Convertir números en palabras
    number_pattern = re.compile(r'\b\d+(\.\d+)?\b')
    text = number_pattern.sub(convert_number_to_words, text)

    # Detectar y convertir expresiones de porcentaje
    text = re.sub(r'(\d+(\.\d+)?)\s*%', lambda match: f"{convert_number_to_words(match)} por ciento", text)

    # Detectar y convertir fechas escritas
    date_pattern = re.compile(r'\b(\d{1,2})\s*de\s*(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de\s*(\d{4})\b', re.IGNORECASE)
    def date_replacement(match):
        day = num2words(int(match.group(1)), lang='es')
        month = match.group(2).lower()
        year = num2words(int(match.group(3)), lang='es')
        return f"{day} de {month} de {year}"
    text = date_pattern.sub(date_replacement, text)

    # Eliminar paréntesis en siglas y mantener siglas en mayúsculas
    text = re.sub(r'\((\w+)\)', r'\1', text)
    text = re.sub(r'\b([A-Z]{2,})\b', lambda match: match.group(0), text)

    # Detectar y convertir horas al formato textual
    time_pattern = re.compile(r'(\d{1,2}):(\d{2})\s*(a\.m\.|p\.m\.|am|pm)', re.IGNORECASE)
    def time_to_words(match):
        hours, minutes, period = int(match.group(1)), int(match.group(2)), match.group(3).replace('.', '').lower()
        hours_word = num2words(hours, lang='es')
        if minutes == 0:  # Si los minutos son cero
            return f"{hours_word} {period}"
        else:  # Si los minutos no son cero
            minutes_word = num2words(minutes, lang='es')
            return f"{hours_word} y {minutes_word} {period}"
    text = time_pattern.sub(time_to_words, text)

    # Eliminar puntos en siglas (ej. "EE.UU." a "EEUU")
    text = re.sub(r'\b([A-Z])\.\s*([A-Z])\.?\b', r'\1\2', text)

    # Eliminar espacios dobles generados
    text = re.sub(r'\s{2,}', ' ', text)

    return text

# Preprocesamiento del texto
#def preprocess_text(text):
#    # Convertir números en palabras
#    number_pattern = re.compile(r'\b\d+(\.\d+)?\b')
#    text = number_pattern.sub(convert_number_to_words, text)
#
#    # Detectar y convertir expresiones de porcentaje (ej. "24.3%" a "veinticuatro punto tres por ciento")
#    text = re.sub(r'(\d+(\.\d+)?)\s*%', lambda match: f"{convert_number_to_words(match)} por ciento", text)
#
#    # Detectar y convertir fechas escritas (ej. "primero de enero de dos mil veinte" a "01/01/2020")
#    date_pattern = re.compile(r'\b(\d{1,2})\s*de\s*(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*de\s*(\d{4})\b', re.IGNORECASE)
#    text = date_pattern.sub(date_replacement, text)
#
#    # Detectar y mantener siglas en mayúsculas (ej. "ONU", "EE.UU.")
#    text = re.sub(r'\b([A-Z]{2,})\b', lambda match: match.group(0), text)
#
#    # Detectar horas en formato hh:mm a.m./p.m. y convertirlas al formato con espacios
#    time_pattern = re.compile(r'(\d{1,2}:\d{2})\s*(a\.m\.|p\.m\.)', re.IGNORECASE)
#    text = time_pattern.sub(lambda match: f"{match.group(1)} {match.group(2).replace('.', '')}", text)
#
#    # Eliminar puntos en siglas
#    text = re.sub(r'\b([A-Z])\.\s*([A-Z])\.?\b', r'\1 \2', text)
#
#    # Detectar y mantener unidades de medida y otros símbolos (ej. "kg", "cm", "%", "$")
#    text = re.sub(r'(\d+)\s*(kg|g|cm|mm|m|km|%)', r'\1 \2', text)
#
#    # Eliminar espacios dobles generados
#    text = re.sub(r'\s{2,}', ' ', text)
#
#    return text

def date_replacement(match):
    day = match.group(1).zfill(2)
    month = months[match.group(2).lower()]
    year = match.group(3)
    return f"{day}/{month}/{year}"
    text = date_pattern.sub(date_replacement, text)

    # Detectar y mantener siglas en mayúsculas (ej. "ONU", "EE.UU.")
    text = re.sub(r'\b([A-Z]{2,})\b', lambda match: match.group(0), text)

    # Detectar unidades de medida y otros símbolos (ej. "kg", "cm", "%", "$")
    text = re.sub(r'(\d+)\s*(kg|g|cm|mm|m|km|%|€|\$)', r'\1 \2', text)

    return text

# Función para comparar oraciones y calcular la similitud máxima
def compare_sentences(provided_text, segment_text):
    best_match = None
    highest_similarity = 0.0
    similarities = []

    # Dividir el texto proporcionado en oraciones
    provided_sentences = nltk.tokenize.sent_tokenize(provided_text)
    print(provided_sentences)

    # Comparar cada oración en `provided_text` con `segment_text`
    for sentence in provided_sentences:
        similarity_ratio = SequenceMatcher(None, sentence.strip(), segment_text.strip()).ratio()
        similarities.append({"oracion": sentence.strip(), "similaridad": similarity_ratio})

        # Guardar la mayor similitud
        if similarity_ratio > highest_similarity:
            highest_similarity = similarity_ratio
            best_match = sentence.strip()

    return highest_similarity

def match_sentences(provided_text, segment_text):
    best_match = None
    highest_similarity = 0.0
    similarities = []

    # Dividir el texto proporcionado en oraciones
    provided_sentences = sent_tokenize(provided_text)

    # Comparar cada oración en `provided_text` con `segment_text`
    for sentence in provided_sentences:
        similarity_ratio = SequenceMatcher(None, sentence.strip(), segment_text.strip()).ratio()
        similarities.append({"oracion": sentence.strip(), "similaridad": similarity_ratio})

        # Guardar la mayor similitud y la oración correspondiente
        if similarity_ratio > highest_similarity:
            highest_similarity = similarity_ratio
            best_match = sentence.strip()

    return best_match

# Función para calcular similitud
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def align_words_to_sentences_2(words, sentences, similarity_threshold_min=0.01, similarity_threshold_max=0.99, step=0.01):
    aligned_segments = []

    # Intentar con diferentes valores de similarity_threshold
    for threshold in reversed([round(x, 2) for x in frange(similarity_threshold_min, similarity_threshold_max, step)]):
        aligned_segments.clear()
        current_sentence_index = 0
        while current_sentence_index < len(sentences):
            current_sentence = sentences[current_sentence_index].split()
            temp_segment = []
            temp_start = None
            temp_end = None
            matched = False

            for word in words:
                label = word["label"].lower()
                if temp_start is None:
                    temp_start = word["start"]
                temp_end = word["end"]
                temp_segment.append(label)

                temp_segment_str = " ".join(temp_segment)
                expected_segment_str = " ".join(current_sentence[:len(temp_segment)])

                similarity = similar(temp_segment_str, expected_segment_str)

                if similarity >= threshold:
                    if len(temp_segment) == len(current_sentence):
                        aligned_segments.append({
                            "sentence": " ".join(current_sentence),
                            "start": temp_start,
                            "end": temp_end
                        })
                        matched = True
                        break
                elif similarity < threshold - 0.1:  # Reiniciar si la similitud cae por debajo de un margen seguro
                    temp_segment = []
                    temp_start = None
                    temp_end = None

            if matched:
                current_sentence_index += 1
            else:
                break  # Salir del bucle si no se puede alinear una oración con este umbral

        if len(aligned_segments) == len(sentences):
            # Si todas las oraciones están alineadas, detener la búsqueda
            print(f"Alignment successful with similarity_threshold = {threshold}")
            break
    else:
        print(f"Warning: Not all sentences could be aligned. Last threshold = {threshold}")

    return aligned_segments

# Función auxiliar para rango flotante
def frange(start, stop, step):
    while start <= stop:
        yield start
        start += step

#def clean_hour_format(sentence):
#    # Patrón para corregir horas con ":cero"
#    sentence = re.sub(r'(\b\w+):cero\s*(am|pm)\b', r'\1 \2', sentence, flags=re.IGNORECASE)
#    # Patrón para corregir horas con ": minutos" (por ejemplo, "cinco: treinta")
#    sentence = re.sub(r'(\b\w+):\s*(\w+)\b', r'\1 \2', sentence, flags=re.IGNORECASE)
#    return sentence

def clean_hour_format(sentence):
    # Convertir covid19 o covid-19 a covid diecinueve
    sentence = re.sub(r'\bcovid[-]?(19)\b', r'covid diecinueve', sentence, flags=re.IGNORECASE)

    # Convertir perú21 o peru21 a peru veintiuno
    def convert_numbers_in_text(match):
        word_part = match.group(1)
        number_part = int(match.group(2))
        return f"{word_part} {num2words(number_part, lang='es')}"

    sentence = re.sub(r'\b(per[úu]?)(\d+)\b', convert_numbers_in_text, sentence, flags=re.IGNORECASE)

    # Corregir horas (ej. cinco: treinta a cinco treinta)
    sentence = re.sub(r'(\b\w+):cero\s*(am|pm)\b', r'\1 \2', sentence, flags=re.IGNORECASE)
    sentence = re.sub(r'(\b\w+):\s*(\w+)\b', r'\1 \2', sentence, flags=re.IGNORECASE)

    return sentence

In [ ]:
# Función para alinear palabras con oraciones del gold_sentences
def align_words_to_sentences(words, sentences, similarity_threshold=0.75):
    aligned_segments = []
    current_sentence_index = 0
    temp_segment = []
    temp_start = None
    temp_end = None

    while current_sentence_index < len(sentences):
        current_sentence = sentences[current_sentence_index].split()  # Lista de palabras de la oración actual
        temp_segment = []  # Resetea el segmento temporal
        temp_start = None
        temp_end = None
        matched = False

        for word in words:
            label = word["label"].lower()
            if temp_start is None:
                temp_start = word["start"]
            temp_end = word["end"]
            temp_segment.append(label)

            # Comparar segmento con oración actual
            temp_segment_str = " ".join(temp_segment)
            expected_segment_str = " ".join(current_sentence[:len(temp_segment)])

            # Verificar similitud con margen de error
            similarity = similar(temp_segment_str, expected_segment_str)

            if similarity >= similarity_threshold:
                if len(temp_segment) == len(current_sentence):
                    # Oración completa encontrada
                    aligned_segments.append({
                        "sentence": " ".join(current_sentence),
                        "start": temp_start,
                        "end": temp_end
                    })
                    matched = True
                    break
            elif not expected_segment_str.startswith(temp_segment_str):
                # Reiniciar si no coincide lo suficiente
                temp_segment = []
                temp_start = None
                temp_end = None

        if matched:
            # Avanzar a la siguiente oración si se encontró coincidencia
            current_sentence_index += 1
        else:
            # Imprimir advertencia para depuración
            print(f"Warning: Sentence not matched -> {sentences[current_sentence_index]}")
            current_sentence_index += 1

    return aligned_segments

In [ ]:
# Ejemplo de texto
example_text = """
Por su parte, el Ministerio de Transportes y Comunicaciones (MTC) y la Autoridad de Transporte Urbano para Lima y Callao (ATU) anunciaron que el horario de la Línea 1 del Metro de Lima será de 5:30 a.m. a 9:00 a.m. y por la tarde de 10:00 a.m. a 8:00 p.m.
"""

# Aplicar preprocesamiento
processed_text = preprocess_text(example_text)
print(processed_text)

In [ ]:
import nltk
import os
import pandas as pd
from pydub import AudioSegment
import torch
import json
from nltk.tokenize import sent_tokenize
from difflib import SequenceMatcher

nltk.download('punkt_tab')

# Ruta del archivo CSV que contiene las rutas de los audios y textos vinculantes
csv_path = "/content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_ATMA_3.csv"

# Leer el archivo CSV
df_input = pd.read_csv(csv_path)

# Procesar cada fila del archivo CSV
for index, row in df_input.iterrows():
    # Obtener las rutas del archivo de audio y del texto
    audio_path = row['ruta_audio']
    print(audio_path)
    text_path = row['ruta_texto_base']
    print(text_path)
    file_json = row['ruta_json']
    print(file_json)

    if os.path.isfile(audio_path) and os.path.isfile(file_json):
        # Extraer el nombre del directorio y archivo sin extensión
        person_value = os.path.basename(os.path.dirname(audio_path))
        audio_name = os.path.splitext(os.path.basename(audio_path))[0]
        text_name = os.path.splitext(os.path.basename(text_path))[0]
        final_name = f"{person_value}_{text_name}"

        # Leer el contenido del archivo de texto
        with open(text_path, 'r', encoding='utf-8') as text_file:
            text_content = text_file.read()

        gold_sentences_original = text_content.strip().split("//")
        # Preprocesar las oraciones del gold_sentences
        gold_sentences = [sentence.strip().lower() for sentence in gold_sentences_original if sentence.strip()]
        gold_sentences = [re.sub(r'[^\w\s:]', '', sentence) for sentence in gold_sentences]  # Eliminar puntuación
        gold_sentences = [preprocess_text(sentence) for sentence in gold_sentences]  # Preprocesar el texto
        gold_sentences = [clean_hour_format(sentence) for sentence in gold_sentences]  # Limpiar formato de horas
        print("Original Sentences:", gold_sentences_original)
        print("Gold Sentences:", gold_sentences)

        # Crear una lista para almacenar los datos de cada palabra
        words_data = []

        # Cargar el archivo de audio completo con pydub
        audio = AudioSegment.from_file(audio_path)

        # Directorio de salida para los chunks de audio segmentados
        output_dir = f"/content/drive/MyDrive/Proyecto_ASR_AG/Aligned_VF_Fix/{final_name}/"
        os.makedirs(output_dir, exist_ok=True)

        with open(file_json, 'r', encoding='utf-8') as json_file:
            json_data = json.load(json_file)

        words = json_data.get("words_cleaned", [])
        print("Words from JSON:", words)

        aligned_segments = align_words_to_sentences_2(words, gold_sentences)

        for idx, gold_sentence in enumerate(gold_sentences_original):
            if (idx > 0):
                # Exportar texto segmentado
                text_filename = f"{output_dir}gold_sentence_{idx}.txt"
                with open(text_filename, 'w', encoding='utf-8') as text_file:
                    text_file.write(gold_sentence)

        # Crear segmentos de audio y exportarlos
        csv_data = []

        for idx, segment in enumerate(aligned_segments):
            print(segment)
            # Extraer segmento de audio
            start_ms = int(segment["start"] * 1000)  # Convertir a milisegundos
            end_ms = int(segment["end"] * 1000)
            audio_segment = audio[start_ms:end_ms]

            # Exportar audio segmentado
            audio_filename = f"{output_dir}segment_{idx + 1}.wav"
            audio_segment.export(audio_filename, format="wav")

            gs_filename = f"{output_dir}gold_sentence_{idx + 1}.txt"

            # Leer el contenido del archivo escrito
            with open(gs_filename, 'r', encoding='utf-8') as text_file:
                    content = text_file.read()  # Leer contenido del archivo
                    #print(f"Contenido del archivo {gs_filename}:")
                    #print(content)  # Imprimir contenido para verificar

            # Exportar texto segmentado
            text_filename = f"{output_dir}segment_{idx + 1}.txt"
            with open(text_filename, 'w', encoding='utf-8') as text_file:
                text_file.write(segment["sentence"])

            # Filtrar palabras asociadas con el segmento actual
            associated_words = [
                word for word in words
                if segment["start"] <= word["start"] <= segment["end"]
            ]

            # Agregar cada palabra al CSV
            for word in associated_words:
                csv_data.append({
                    "audio_base":audio_path,
                    "audio_file": audio_filename,
                    "sentence_file": text_filename,
                    "gold_sentence_path": gs_filename,
                    "gold_sentence": content,
                    "sentence": segment["sentence"],
                    "sentence_start": segment["start"],
                    "sentence_end": segment["end"],
                    "word": word["label"],
                    "word_start": word["start"],
                    "word_end": word["end"]
                })

        # Guardar en un CSV
        output_csv_path = f"/content/drive/MyDrive/Proyecto_ASR_AG/Aligned_VF_Fix/{final_name}.csv"
        df_output = pd.DataFrame(csv_data)
        df_output.to_csv(output_csv_path, index=False, encoding='utf-8')

    print(f"Segmented audios and CSV saved successfully at {output_csv_path}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


/content/drive/MyDrive/Proyecto_ASR_AG/Proyecto_AG/ATMA/2020-05-27_03_54_34.wav
/content/drive/MyDrive/Proyecto_ASR_AG/Texto_Base/Lectura_005.txt
/content/drive/MyDrive/Proyecto_ASR_AG/Proyecto_AG/ATMA/2020-05-27_03_54_34.wav.json
Original Sentences: ['', ' Motociclistas anuncian marcha nacional ', ' Saldrán a las calles a protestar contra el proyecto de ley que prohíbe el desplazamiento de estos vehículos con dos o más ocupantes. ', ' La Asociación de Motociclistas del Perú anunció una marcha nacional para este jueves 11 de julio. ', ' Esto luego de que la Comisión de Defensa Nacional y Orden Interno del Congreso de la República aprobara el proyecto de ley que prohíbe el desplazamiento de una motocicleta con dos o más ocupantes. ', ' El proyecto de ley, que es impulsado por el congresista Clayton Galván de Cambio 21, también obliga a los conductores a usar casco con el número de la placa de la moto. ', ' Según el parlamentario, “la medida se justifica por la gran cantidad de delitos q